In [6]:
import pandas as pd
import boto3
import os

In [18]:
# Read the data connection variables
key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

In [19]:
# Create connection
s3_client = boto3.client(
    "s3",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint,
    use_ssl=True
)

In [25]:
os.makedirs("data", exist_ok=True)

s3_client.download_file(bucket_name, '/data/sensors_data.csv', 'data/sensors_data.csv')
s3_client.download_file(bucket_name, '/data/measured_value.csv', 'data/measured_value.csv')

In [21]:
days = 7

In [22]:
sensors_data = pd.read_csv('data/sensors_data.csv')
new_sensors_data = sensors_data.drop(index=range(days)).reset_index(drop=True)
new_sensors_data.to_csv('upload_data/sensors_data.csv', sep=',', index=False, header=True)

In [23]:
measured_value = pd.read_csv('data/measured_value.csv')
new_measured_value = measured_value.drop(index=range(days)).reset_index(drop=True)
new_measured_value.to_csv('upload_data/measured_value.csv', sep=',', index=False, header=True)

In [24]:
# Upload the data
s3_client.upload_file('upload_data/historical.csv', bucket_name, Key="data/historical.csv")
s3_client.upload_file('upload_data/sensors_data.csv', bucket_name, Key="data/sensors_data.csv")
s3_client.upload_file('upload_data/measured_value.csv', bucket_name, Key="data/measured_value.csv")